# Workgroup5

- Janice Mamani

In [11]:
using Pkg
Pkg.add(["CSV", "DataFrames", "Plots", "StatsPlots", "GLM", "Distributions", "Random", "ScikitLearn", "Econometrics", "StatsBase", "StatsModels"])


# Load the necessary packages
using CSV
using DataFrames
using Plots
using StatsPlots
using GLM
using Distributions
using Random
using ScikitLearn
using ScikitLearn.CrossValidation: train_test_split
using ScikitLearn.LinearModel: LassoCV
using StatsBase
using StatsModels
using Statistics
using GLM


   Resolving package versions...
  No Changes to `~/.julia/environments/v1.8/Project.toml`
  No Changes to `~/.julia/environments/v1.8/Manifest.toml`


LoadError: UndefVarError: LinearModel not defined

In [15]:
# Load the data
df = CSV.read("processed_esti.csv", DataFrame)

Row,y,w,gender_female,gender_male,gender_transgender,ethnicgrp_asian,ethnicgrp_black,ethnicgrp_mixed_multiple,ethnicgrp_other,ethnicgrp_white,partners1,postlaunch,msm,age,imd_decile
,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64
1,1,1,0,1,0,0,0,1,0,0,0,1,0,27,5
2,0,0,0,1,0,0,0,0,0,1,0,0,0,19,6
3,0,1,0,1,0,0,1,0,0,0,0,1,0,26,4
4,0,0,1,0,0,0,0,0,0,1,1,0,0,20,2
5,1,1,1,0,0,1,0,0,0,0,0,1,0,24,3
6,1,1,0,1,0,0,0,0,0,1,0,1,0,24,2
7,1,1,1,0,0,0,0,0,0,1,0,0,0,24,4
8,0,0,0,1,0,0,1,0,0,0,0,1,0,21,2
9,0,1,0,1,0,0,0,0,0,1,1,0,0,27,2


In [13]:
variables = [
    "y", "w", "gender_female", "gender_male", "gender_transgender",
    "ethnicgrp_asian", "ethnicgrp_black", "ethnicgrp_mixed_multiple",
    "ethnicgrp_other", "ethnicgrp_white", "partners1", "postlaunch",
    "age", "imd_decile"
]


14-element Vector{String}:
 "y"
 "w"
 "gender_female"
 "gender_male"
 "gender_transgender"
 "ethnicgrp_asian"
 "ethnicgrp_black"
 "ethnicgrp_mixed_multiple"
 "ethnicgrp_other"
 "ethnicgrp_white"
 "partners1"
 "postlaunch"
 "age"
 "imd_decile"

In [22]:
# Dividir en grupos de control y tratamiento
control_group = filter(row -> row[:w] == 0, df)
treatment_group = filter(row -> row[:w] == 1, df)


Row,y,w,gender_female,gender_male,gender_transgender,ethnicgrp_asian,ethnicgrp_black,ethnicgrp_mixed_multiple,ethnicgrp_other,ethnicgrp_white,partners1,postlaunch,msm,age,imd_decile
,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64
1,1,1,0,1,0,0,0,1,0,0,0,1,0,27,5
2,0,1,0,1,0,0,1,0,0,0,0,1,0,26,4
3,1,1,1,0,0,1,0,0,0,0,0,1,0,24,3
4,1,1,0,1,0,0,0,0,0,1,0,1,0,24,2
5,1,1,1,0,0,0,0,0,0,1,0,0,0,24,4
6,0,1,0,1,0,0,0,0,0,1,1,0,0,27,2
7,1,1,1,0,0,0,1,0,0,0,0,0,0,21,6
8,0,1,0,1,0,0,0,0,0,1,0,0,1,18,4
9,0,1,0,1,0,0,0,0,0,1,1,1,1,26,2


In [26]:
using DataFrames
using Statistics
using GLM
using StatsModels
using Plots

In [28]:
function calculate_stats(variable)
    control_mean = mean(control_group[!, variable])
    control_std = std(control_group[!, variable])

    treatment_mean = mean(treatment_group[!, variable])
    treatment_std = std(treatment_group[!, variable])

    return control_mean, control_std, treatment_mean, treatment_std
end

calculate_stats (generic function with 1 method)

In [32]:
balance_table = DataFrame(
    Variable = String[],
    Control_mean = Float64[],
    Control_std = Float64[],
    Treatment_mean = Float64[],
    Treatment_std = Float64[],
)

for var in variables
    control_mean, control_std, treatment_mean, treatment_std = calculate_stats(var)
    push!(balance_table, (var, control_mean, control_std, treatment_mean, treatment_std))
end

# Mostrar la tabla de balance
println(balance_table)


14×5 DataFrame
 Row │ Variable                  Control_mean  Control_std  Treatment_mean  Treatment_std 
     │ String                    Float64       Float64      Float64         Float64       
─────┼────────────────────────────────────────────────────────────────────────────────────
   1 │ y                           0.211491      0.408616       0.476656        0.499726
   2 │ w                           0.0           0.0            1.0             0.0
   3 │ gender_female               0.580685      0.493749       0.587405        0.492569
   4 │ gender_male                 0.418093      0.493547       0.409338        0.491979
   5 │ gender_transgender          0.00122249    0.0349642      0.00325733      0.0570109
   6 │ ethnicgrp_asian             0.0550122     0.228144       0.0716612       0.258066
   7 │ ethnicgrp_black             0.0929095     0.290483       0.0803474       0.271978
   8 │ ethnicgrp_mixed_multiple    0.0929095     0.290483       0.0846906       0.278572
   9

In [40]:
default(size=(1200, 600))

group_labels = ["Control", "Treated"]
colors = [:blue, :red]

mean_y = combine(groupby(df, :w), :y => mean)

bar(mean_y[!,:w], mean_y[!,:y_mean], group=mean_y[!,:w], bar_width=0.8, fillalpha=0.6, label="", color=palette(colors))

xlabel!("Group (0 = Control, 1 = Treated)")
ylabel!("Proportion of STI Testing")
title!("Proportion of STI Testing (Outcome y) in Treated and Control Groups")

display(plot)



plot (generic function with 4 methods)